In [1]:
import sys
sys.path.append("../")

In [2]:
import numpy as np
import pandas as pd
from sklearn import datasets, model_selection, ensemble, svm
from modules.feature_explain import FeatureExplainer

In [3]:
iris = datasets.load_iris()

In [4]:
iris.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [5]:
X = iris["data"]

In [6]:
y = iris["target"]

In [7]:
col_names = iris["feature_names"]

In [8]:
X = pd.DataFrame(X, columns=col_names)

In [9]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
X_train.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
96,5.7,2.9,4.2,1.3
105,7.6,3.0,6.6,2.1
66,5.6,3.0,4.5,1.5
0,5.1,3.5,1.4,0.2
122,7.7,2.8,6.7,2.0


In [11]:
rf = ensemble.RandomForestClassifier()
svc = svm.SVC(probability=True)

In [12]:
rf.fit(X_train, y_train)
svc.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [13]:
print("SVM", svc.score(X_test, y_test))
print("RF", rf.score(X_test, y_test))

SVM 1.0
RF 0.98


In [14]:
exp_rf_lime = FeatureExplainer("lime", "tabular", rf, data=X_train)

In [15]:
exp_rf_lime.explain(X_test.iloc[0])

[('0.30 < petal width (cm) <= 1.30', 0.20096180603927055),
 ('4.25 < petal length (cm) <= 5.10', 0.19603126462414702),
 ('5.75 < sepal length (cm) <= 6.32', 0.03124173509285905),
 ('2.77 < sepal width (cm) <= 3.00', -0.012637010774709299)]

In [16]:
exp_rf_lime.explain(X_test.iloc[0:3])

[[('0.30 < petal width (cm) <= 1.30', 0.21341475322206355),
  ('4.25 < petal length (cm) <= 5.10', 0.19277142326313856),
  ('2.77 < sepal width (cm) <= 3.00', -0.014683477261293106),
  ('5.75 < sepal length (cm) <= 6.32', 0.012437966992397891)],
 [('1.60 < petal length (cm) <= 4.25', 0.23707793447639072),
  ('petal width (cm) <= 0.30', -0.1663358881198918),
  ('5.10 < sepal length (cm) <= 5.75', 0.022597536671949487),
  ('sepal width (cm) > 3.30', 0.0007048577133007474)],
 [('petal length (cm) > 5.10', -0.24582153612950597),
  ('petal width (cm) > 1.80', -0.2111575069132927),
  ('sepal width (cm) <= 2.77', -0.018086088666704626),
  ('sepal length (cm) > 6.32', -0.014765139513410803)]]

In [17]:
exp_svc_lime = FeatureExplainer("lime", "tabular", svc, data=X_train)

In [18]:
exp_rf_lime.explain(X_test.iloc[0])

[('0.30 < petal width (cm) <= 1.30', 0.21067984889641855),
 ('4.25 < petal length (cm) <= 5.10', 0.20423587533561863),
 ('5.75 < sepal length (cm) <= 6.32', 0.02635865252167193),
 ('2.77 < sepal width (cm) <= 3.00', -0.005312750920747935)]

In [19]:
exp_rf_shap = FeatureExplainer("shap", "tabular", rf, shap_method="auto", data=X_train)

In [20]:
exp_rf_shap.shap_method # because of RandomForest

'tree'

In [21]:
exp_rf_shap.data # has been summarized

array([[5.75, 3.  , 4.25, 1.3 ]])

In [22]:
exp_rf_shap.explain(X_test.iloc[0])

[array([0., 0., 0., 0.]),
 array([ 0.00333333, -0.01166667, -0.01166667,  0.        ]),
 array([-0.00333333,  0.01166667,  0.01166667,  0.        ])]

In [23]:
exp_svc_shap = FeatureExplainer("shap", "tabular", svc, shap_method="auto", data=X_train)

In [24]:
svc.predict_proba

<bound method BaseSVC._predict_proba of SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)>

In [27]:
shap_values = exp_svc_shap.explain(X_test.iloc[0])

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!


In [26]:
import shap

In [34]:
shap.force_plot(exp_svc_shap.data, shap_values[0])

Exception: In v0.20 force_plot now requires the base value as the first parameter! Try shap.force_plot(explainer.expected_value, shap_values) or for multi-output models try shap.force_plot(explainer.expected_value[0], shap_values[0]).

In [30]:
exp_svc_shap.data

array([[5.75, 3.  , 4.25, 1.3 ]])